In [ ]:
# !pip install vllm==0.10.2
# !pip install pyarrow

In [ ]:
%%writefile gen_answer.py
import re
import torch
import pandas as pd
import ast
import vllm

def main():
    # Model path
    model_path = "/mnt/nfs-mnj-hot-99/tmp/hokuyama/models/Qwen3-14B"

    # Initialize the LLM
    llm = vllm.LLM(
        model_path,
        tensor_parallel_size=torch.cuda.device_count(),
        gpu_memory_utilization=0.99,
        trust_remote_code=True,
        dtype="half",
        enforce_eager=True,
        max_model_len=4096,
        max_num_seqs=32,
        disable_log_stats=True,
        enable_prefix_caching=True,
    )
    tokenizer = llm.get_tokenizer()

    SYSTEM_PROMPT = """You are an expert content moderator AI model. 
Your task is to analyze a Reddit comment against a specific community rule and predict the probability that the comment violates that rule.
The probability must be a float value between 0.00 and 1.00, formatted to two decimal places.
A probability closer to 1.00 indicates a high likelihood of a rule violation, and a probability closer to 0.00 indicates a low likelihood of a rule violation.

YOUR OUTPUT MUST BE ONLY THE PROBABILITY VALUE, WITH NO OTHER TEXT, REASONING, OR EXPLANATION."""

    USER_PROMPT_TEMPLATE = """### RULE
{rule_text}

### COMMENT TO EVALUATE
{comment_body}"""

    # Read data
    df_train = pd.read_csv("jigsaw-agile-community-rules/train_v2.csv")

    prompts = []
    for i, r in df_train.iterrows():
        user_prompt = USER_PROMPT_TEMPLATE.format(
            rule_text=r['rule'],
            comment_body=r['body']
        )
        messages = [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': user_prompt}
        ]
        prompts.append(messages)

    outputs = llm.chat(
        prompts,
        vllm.SamplingParams(
            seed=0,
            skip_special_tokens=True,
            max_tokens=1024,
            temperature=0,
            repetition_penalty=1.05,
        ),
        # chat_template_kwargs={"enable_thinking": False},
        use_tqdm=True
    )
    responses =  [output.outputs[0].text for output in outputs]

    answers = []
    for response in responses:
        try:
            answer = float(response.split("</think>")[-1])
            answers.append(answer)
        except:
            answers.append(0.5)

    df_train["llm_output"] = answers

    # Save as Parquet file
    df_train.to_parquet('output.parquet', index=False)

if __name__ == "__main__":
    main()

In [ ]:
!python gen_answer.py

In [ ]:
import pandas as pd
df = pd.read_parquet("output.parquet")
df["rule_violation"] = df["llm_output"]
df[["row_id", "rule_violation"]].to_csv("submission.csv", index=False)

In [ ]:
from sklearn.metrics import roc_auc_score
df = pd.read_parquet("output.parquet")
roc_auc_score(df["rule_violation"], df["llm_output"])

In [ ]:
df

In [ ]:
df = pd.read_csv("jigsaw-agile-community-rules/train.csv")

# 1. body と rule のユニーク値を取得
unique_bodies = df['body'].drop_duplicates()
unique_rules = df['rule'].drop_duplicates()

# 2. 全 body × 全 rule の組み合わせ（重複なし）
comb = pd.MultiIndex.from_product([unique_bodies, unique_rules], names=['body', 'rule'])
mapped_df = pd.DataFrame(index=comb).reset_index()

# 3. (body, rule)ごとに最頻値（mode）を求める関数
def get_mode(s):
    m = s.mode()
    if m.empty:
        return None  # ここは None でOK
    else:
        return m.iloc[0]

# 4. 元dfから body, rule ごとの rule_violation の mode を抽出
rule_violation_map = (
    df.groupby(['body', 'rule'])['rule_violation']
      .agg(get_mode)
      .reset_index()
)

# 5. 全組み合わせ dataframe に rule_violation を結合
result = mapped_df.merge(rule_violation_map, on=['body', 'rule'], how='left')

# 6. rule_violation が NaN（ない場合）は 0.5 にする
result['rule_violation'] = result['rule_violation'].fillna(0.5)

In [ ]:
result

In [ ]:
result[result["rule_violation"] != 0.5].reset_index(drop=True).reset_index(drop=False).to_csv("jigsaw-agile-community-rules/train_v2.csv", index=False)